https://mlflow.org/docs/latest/python_api/mlflow.html

In [1]:
!pip install mlflow --quiet

In [3]:
!git clone https://github.com/RiskModellingResearch/DeepLearning_Autumn22

Cloning into 'DeepLearning_Autumn22'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 49 (delta 13), reused 44 (delta 8), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [4]:
import mlflow

Отдельный объект создавать не нужно, просто вызываем необходимые методы из библиотеки

В папке проекта, где лежит запускаемый код, создаться папка `mlruns` и там будут лежать папки, которые будут хранить все логи по всем запускам

В терминале, будучи в папке, где лежит папка `mlruns` запускаем в консоли 

`mlflow ui`

<img src="https://github.com/RiskModellingResearch/DeepLearning_Winter22/blob/main/week_04/images/mlflow.png?raw=1" alt="Drawing" />

# Пример

In [5]:
!pip install torchmetrics --quiet

In [6]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import torch

print(torch.__version__)

import torch.nn as nn # содержит функции для реалзации архитектуры нейронных сетей
import torch.optim as optim
import torch.utils.data as data_utils

from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import mlflow

1.12.1+cu113


In [7]:
INPUT_SIZE = 37
HIDDEN_SIZE = 25
OUTPUT_SIZE = 4
LEARNING_RATE = 1e-2
EPOCHS = 400
BATCH_SIZE = 256

## ETL

In [8]:
def load_dataset():
    X = pd.read_csv('./DeepLearning_Autumn22/week_04/data/X_cat.csv', sep='\t', index_col=0)
    target = pd.read_csv('./DeepLearning_Autumn22/week_04/data/y_cat.csv', sep='\t', index_col=0, names=['status'])  # header=-1,

    print(X.shape)
    display(X.head())

    target = target.iloc[:, :].values
    target[target == 'Died'] = 'Euthanasia'

    le = LabelEncoder()
    y = le.fit_transform(target)

    return X, y

In [9]:
def create_data_loader(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X.values, y,
                                                        test_size=0.2, stratify=y, random_state=42)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    train_tensor = data_utils.TensorDataset(torch.tensor(X_train.astype(np.float32)), torch.tensor(y_train))
    train_loader = data_utils.DataLoader(dataset=train_tensor,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True)

    test_tensor = data_utils.TensorDataset(torch.tensor(X_test.astype(np.float32)), torch.tensor(y_test))
    test_loader = data_utils.DataLoader(dataset=test_tensor,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False)
    
    return X_train, X_test, y_train, y_test, train_loader, test_loader

## Model Architecture

In [10]:
class MLPNet(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super(MLPNet, self).__init__()

        self.linear1 = torch.nn.Linear(input_size, hidden_size)

        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)

        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output = self.linear1(x)
        output = torch.relu(output)

        output = self.linear2(output)
        output = torch.relu(output)

        output = self.linear3(output)
        predictions = torch.softmax(output, dim=1)

        return predictions

## Train pipeline

In [11]:
def run_train(train_loader, test_loader):
    model = MLPNet(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE)

    criterion = nn.CrossEntropyLoss()
    accuracy = Accuracy()
    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

    step = 0
    for epoch in range(EPOCHS):
        model.train()

        for features, label in train_loader:
            # Reset gradients
            optimizer.zero_grad()

            output = model(features)
            # Calculate error and backpropagate
            loss = criterion(output, label)
            loss.backward()
            acc = accuracy(output, label).item()

            # Update weights with gradients
            optimizer.step()

            mlflow.log_metric('Train/CrossEntropyLoss', loss.item(), step)
            mlflow.log_metric('Train/Accuracy', acc, step)

            step += 1

            if step % 50 == 0:
                print('EPOCH %d STEP %d : train_loss: %f train_acc: %f' %
                      (epoch, step, loss, acc))


        # Run validation
        running_loss = []
        valid_scores = []
        valid_labels = []
        model.eval()
        with torch.no_grad():
            for features, label in test_loader:
                output = model(features)
                # Calculate error and backpropagate
                loss = criterion(output, label)

                running_loss.append(loss.item())
                valid_scores.extend(torch.argmax(output, dim=1))
                valid_labels.extend(label)

        valid_accuracy = accuracy(torch.tensor(valid_scores), torch.tensor(valid_labels)).item()

        mlflow.log_metric('Valid/CrossEntropyLoss', np.mean(running_loss), step)
        mlflow.log_metric('Valid/Accuracy', valid_accuracy, step)

        print('EPOCH %d : valid_loss: %f valid_acc: %f' % (epoch, np.mean(running_loss), valid_accuracy))

    return

In [12]:
def run_experiment():
    features, labels = load_dataset()
    X_train, X_test, y_train, y_test, train_loader, test_loader = create_data_loader(features, labels)

    mlflow.log_param("LEARNING_RATE", LEARNING_RATE)
    mlflow.log_param("INPUT_SIZE", INPUT_SIZE)
    mlflow.log_param("HIDDEN_SIZE", HIDDEN_SIZE)
    mlflow.log_param("NROF_CLASSES", OUTPUT_SIZE)
    mlflow.log_param("BATCH_SIZE", BATCH_SIZE)

    run_train(train_loader, test_loader)

    return

In [13]:
# For Google Colab

# # run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 80 &")# run tracking UI in the background

# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6
!pip install pyngrok --quiet
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
public_url = ngrok.connect(port=5000, proto="http", options={"bind_tls": True})
print("MLflow Tracking UI:", public_url)

MLflow Tracking UI: NgrokTunnel: "http://5edf-34-86-5-33.ngrok.io" -> "http://localhost:80"


In [ ]:
with mlflow.start_run(run_name='MLFlow_experiment'):
    # mlflow.log_artifact("./DeepLearning_Autumn22/week_04/MLFlow.ipynb")
    run_experiment()  

(26729, 37)
   IsDog    Age  HasName  NameLength  NameFreq  MixColor  ColorFreqAsIs  \
0      1  365.0        1           7  0.000157         1       0.032919   
1      0  365.0        1           5  0.000655         0       0.008092   
2      1  730.0        1           6  0.000052         1       0.026293   
3      0   21.0        0           7  0.285871         0       0.000471   
4      1  730.0        0           7  0.285871         0       0.023831   

   ColorFreqBase  TabbyColor  MixBreed  ...  SexStatus_Flawed  \
0       0.463624           0         1  ...                 1   
1       0.015005           1         1  ...                 1   
2       0.357521           0         1  ...                 1   
3       0.058418           0         1  ...                 0   
4       0.075353           0         0  ...                 1   

   SexStatus_Intact  SexStatus_Unknown  Weekday_0  Weekday_1  Weekday_2  \
0                 0                  0          0          0          1